In [1]:
from typing_extensions import TypedDict
from typing import List
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from pydantic import BaseModel

llm = init_chat_model("openai:gpt-5-mini-2025-08-07")

In [ ]:
class State(TypedDict):
    dish: str
    ingredients: list[dict]
    recipe_steps: str
    plating_instructions: str


class Ingredient(BaseModel):
    name: str
    quantity: str
    unit: str


class IngredientsOutput(BaseModel):
    ingredients: list[Ingredient]


In [ ]:
def list_ingredients(state: State) -> State:
    structured_output = llm.with_structured_output(IngredientsOutput)
    response = structured_output.invoke(f"List 5-8 ingredients for {state['dish']}")

    return {"ingredients": response.ingredients}


def create_recipe(state: State) -> State:
    response = llm.invoke(
        f"Create a recipe for {state['dish']} with the following ingredients: {state['ingredients']}"
    )

    return {"recipe_steps": response.content}


def describe_plating(state: State) -> State:
    response = llm.invoke(
        f"Describe the plating for {state['dish']} with the following recipe: {state['recipe_steps']}"
    )

    return {"plating_instructions": response.content}


In [4]:
graph_builder = StateGraph(State)

graph_builder.add_node("list_ingredients", list_ingredients)
graph_builder.add_node("create_recipe", create_recipe)
graph_builder.add_node("describe_plating", describe_plating)

graph_builder.add_edge(START, "list_ingredients")
graph_builder.add_edge("list_ingredients", "create_recipe")
graph_builder.add_edge("create_recipe", "describe_plating")
graph_builder.add_edge("describe_plating", END)

graph = graph_builder.compile()


In [5]:
graph.invoke({"dish": "pizza"})

{'dish': 'pizza',
 'ingredients': [Ingredient(name='Pizza dough', quantity='1', unit='ball'),
  Ingredient(name='Tomato sauce', quantity='1/2', unit='cup'),
  Ingredient(name='Mozzarella cheese (shredded)', quantity='8', unit='oz'),
  Ingredient(name='Olive oil', quantity='1', unit='tbsp'),
  Ingredient(name='Garlic (minced)', quantity='2', unit='cloves'),
  Ingredient(name='Fresh basil', quantity='6', unit='leaves'),
  Ingredient(name='Salt', quantity='1', unit='tsp'),
  Ingredient(name='Dried oregano', quantity='1', unit='tsp')]}